In [1]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

CUDA available: True
Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 4060 Laptop GPU
GPU 0 memory allocated: 0.00 GB
GPU 0 memory reserved: 0.00 GB


In [2]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType, Make_Output_Dir_JF, Make_Analysis_Dir_JF, \
                                    ConsolidateImageShapes, TrimZSlices, AutomaticSpotDetection_JF, NativeDataType

from src.SequentialSteps import CellSegmentationStepClass_JF, BIGFISH_SpotDetection, SimpleCellposeSegmentaion, CellProperties, DilationedCytoMask ,\
                                    DisplaySequentialParams, IlluminationCorrection

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.GeneralOutput import OutputClass

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


In [3]:
scope = ScopeClass() 
data = DataContainer()
settings = Settings(name='demo')
experiment = Experiment()

In [4]:
# you can change the parameters
experiment.initial_data_location = ['smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_0min_20220224']
                                    # 'smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_10min_20220224']

# data.local_dataset_location = [r'C:\Users\formanj\GitHub\FISH_Processing\Demos\DUSP1_Dex_10min_20220224', 
#                                  r'C:\Users\formanj\GitHub\FISH_Processing\Demos\DUSP1_Dex_0min_20220224']
experiment.FISHChannel = 0
experiment.nucChannel = 2
experiment.cytoChannel = None
experiment.voxel_size_z = 500

settings.num_chunks_to_run = 1 # this is the limit to how many chunks you want to run and it will stop after that
settings.psuedoCyto = 1
settings.sigma_dict = {0: 50, 1: 100, 2: 100}

scope.spot_yx = 130
scope.spot_z = 360
scope.voxel_size_yx = 100

Parameters.get_parameters()

{'voxel_size_yx': 100,
 'spot_z': 360,
 'spot_yx': 130,
 'microscope_saving_format': 'pycromanager',
 'local_dataset_location': None,
 'h5_file': None,
 'total_num_chunks': None,
 'images': None,
 'masks': None,
 'name': 'demo',
 'return_data_to_NAS': True,
 'NUMBER_OF_CORES': 4,
 'save_files': True,
 'num_chunks_to_run': 1,
 'download_data_from_NAS': True,
 'connection_config_location': 'c:\\Users\\Jack\\Documents\\GitHub\\FISH_Processing\\config_nas.yml',
 'share_name': 'share',
 'display_plots': True,
 'load_in_mask': False,
 'psuedoCyto': 1,
 'sigma_dict': {0: 50, 1: 100, 2: 100},
 'initial_data_location': ['smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_0min_20220224'],
 'index_dict': None,
 'nucChannel': 2,
 'cytoChannel': None,
 'FISHChannel': 0,
 'voxel_size_z': 500,
 'independent_params': None,
 'kwargs': None,
 'timestep_s': None}

In [5]:
Parameters.validate()

None
cytoChannel not set


In [6]:
settings.load_in_mask = True
NativeDataType().run()

Overwriting independent_params in Experiment
Adding leftover kwargs to Settings
Adding position_indexs to Settings


In [7]:
# this will load in your data
print(data)

DataContainer:
local_dataset_location: ['c:\\Users\\Jack\\Documents\\GitHub\\FISH_Processing\\dataBases\\DUSP1_Dex_0min_20220224\\DUSP1_Dex_0min_20220224.h5'] 
h5_file: [<HDF5 file "DUSP1_Dex_0min_20220224.h5" (mode r)>] 
total_num_chunks: 30 
images: dask.array<rechunk-merge, shape=(30, 1, 3, 27, 936, 640), dtype=float32, chunksize=(1, 1, 3, 27, 936, 640), chunktype=numpy.ndarray> 
masks: dask.array<rechunk-merge, shape=(30, 1, 3, 1, 936, 640), dtype=float32, chunksize=(1, 1, 3, 1, 936, 640), chunktype=numpy.ndarray> 



In [8]:
IlluminationCorrection().run()

Processing channel 0 with sigma=50...
Processing channel 1 with sigma=100...
Processing channel 2 with sigma=100...


NotImplementedError: Percentiles only implemented for 1-d arrays

In [ ]:
# if you would like to remove the original data from disk
remove_local_data_but_keep_h5().run()

In [ ]:

# settings.cellpose_min_size = 500
# settings.cellpose_diameter = [180, 90]
# settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

# SimpleCellposeSegmentaion().run()

In [ ]:
# DilationedCytoMask().run()

In [ ]:
# DisplaySequentialParams().run()

In [ ]:
# CellProperties().run()

In [ ]:
inst = OutputClass.get_all_instances()
vars(inst[-1]).keys()

In [ ]:
# inst[-1].cell_properties.columns

In [ ]:
# print(inst[-1].cell_properties['nuc_label'])
# print(inst[-1].cell_properties['cell_label'])
# print(inst[-1].cell_properties['cyto_label'])


In [ ]:
# Display().displayMask(channel=1, label=5)

In [ ]:
Save_Parameters().run()